In [ ]:
%pip install pandas
%pip install folium
%pip install mapclassify
%pip install geopandas
%pip install selenium

In [2]:
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import display

In [3]:
# Load election data
election_data = pd.read_csv("data/election_results/kombinirani_rezultati.csv")

# Load GeoJSON data
geojson_data = gpd.read_file("data/administrative_units/gradovi_opcine_zupanije.geojson")

In [ ]:
print(election_data.head())
print(geojson_data.head())

# Ensure that the key columns are named consistently
election_data['Županija'] = election_data['Županija'].str.strip()  # Strip any extra spaces
election_data['Grad/općina'] = election_data['Grad/općina/država'].str.strip()

geojson_data['Županija'] = geojson_data['text_right'].str.strip()
geojson_data['Grad/općina'] = geojson_data['text_left'].str.strip()

# Merge dataframes based on the matching columns
parlamentarni_izbori = geojson_data.merge(election_data, on=['Županija', 'Grad/općina'], how='inner')

In [ ]:
# Assuming parlamentarni_izbori is your already loaded GeoDataFrame
# Calculate the percentage of Važeći listići divided by Ukupno birača
parlamentarni_izbori['percentage'] = (parlamentarni_izbori['Važeći listići'] / parlamentarni_izbori['Ukupno birača']) * 100

min_value = parlamentarni_izbori['percentage'].min()
max_value = parlamentarni_izbori['percentage'].max()

# Create a color map for the percentage (from blue to yellow to red)
colormap = folium.LinearColormap(
    colors=['blue','yellow','red'],
    vmin=min_value,  # Minimum value for color map
    vmax=max_value,  # Maximum value for color map
    caption='Percentage of Važeći listići to Ukupno birača'
)

# Add color column based on percentage
parlamentarni_izbori['color'] = parlamentarni_izbori['percentage'].apply(lambda x: colormap(x))

# Convert Timestamp columns to strings
for col in parlamentarni_izbori.select_dtypes(include=['datetime64[ns, UTC]', 'datetime64[ns]']).columns:
    parlamentarni_izbori[col] = parlamentarni_izbori[col].astype(str)

# Prepare hover information
parlamentarni_izbori['hover_info'] = parlamentarni_izbori.apply(
    lambda row: f"Županija: {row['Županija']}<br>Grad/Općina: {row['Grad/općina']}<br>Ukupno birača: {row['Ukupno birača']}<br>Važeći listići: {row['Važeći listići']}<br>Percentage: {row['percentage']:.2f}%" +
    ''.join([f"<br>{col}: {row[col]}" for col in parlamentarni_izbori.columns if col.endswith('%')]), axis=1
)

# Create a folium map centered on Croatia
m = folium.Map(location=[45.1, 15.2], zoom_start=7)

# Function to style each feature based on percentage
def style_function(feature):
    return {
        'fillColor': feature['properties']['color'],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5
    }

# Add GeoJSON layer with hover functionality
folium.GeoJson(
    parlamentarni_izbori,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['hover_info'], 
        aliases=['Info: '],
        localize=True
    )
).add_to(m)

# Add the colormap to the map
colormap.add_to(m)

# Display the map directly in the notebook
display(m)
